In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from importlib import reload
from RelationalModule import test_env
from RelationalModule import test_agent_sandbox as train

ImportError: cannot import name 'test_env' from 'RelationalModule' (unknown location)

In [ ]:
reload(test_env)

In [ ]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [9,9]

# All game parameters
game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=50, greyscale_state=True)

env = test_env.Sandbox(**game_params)

In [ ]:
env.enc_to_grey()

### Random non-overlapping goal and initial location

Extract 2 numbers in x*y and then decode them as (x,y)

In [ ]:
def random_start(X=10, Y=10):
    s1, s2 = np.random.choice(X*Y, 2, replace=False)
    initial = [s1//X, s1%X]
    goal = [s2//X, s2%X]
    return initial, goal

random_start()